In [45]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

import psycopg2 as pg
import os


with open('open_api_key.txt', 'r') as f:
    openai_key = f.read().strip()

In [46]:
model = ChatOpenAI(model="gpt-4", api_key=openai_key)


In [47]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [48]:
# to take the github repo data
import requests
from datetime import datetime, timedelta
from pprint import pprint
import json
import psycopg2 as pg
from psycopg2 import sql

def github_api_request(owner, repo, endpoint, params={}):
    if endpoint:
        base_url = f'https://api.github.com/repos/{owner}/{repo}/{endpoint}'
    else:
        base_url = f'https://api.github.com/repos/{owner}/{repo}'
    response = requests.get(base_url, params=params)
    return response.json()

def count_commits(owner, repo, params={}):
    commits = github_api_request(owner, repo, 'commits', params)
    return len(commits)

def count_issues(owner, repo, params={}):
    issues_data = github_api_request(owner, repo, 'issues', params)
    issue_count = len([issue for issue in issues_data if 'pull_request' not in issue])
    return issue_count

def count_pull_requests(owner, repo, params={}):
    pulls = github_api_request(owner, repo, 'pulls', params)
    return len(pulls)

def count_watcher(owner, repo, parmas={}):
    repo = github_api_request(owner, repo, '')
    # pprint(repo)
    return repo['watchers_count']

def connect_to_db(host, port, user, passwd):
    conn = pg.connect(
        host=host,
        port=port,
        database='your_database',
        user=user,
        password=passwd
    )
    return conn


def insert_to_db(conn, table : str, data : dict):

    cur = conn.cursor()
    insert_query = sql.SQL("INSERT INTO {table} ({columns}) VALUES ({values})").format(
        table=sql.Identifier(table),
        columns=sql.SQL(', ').join(map(sql.Identifier, data.keys())),
        values=sql.SQL(', ').join(map(sql.Literal, data.values()))
    )
    cur.execute(insert_query)
    conn.commit()
    cur.close()

In [49]:
# set the owner and repo
owner = 'octocat'
repo = 'Hello-World'

# get the diff between today and month
today = datetime.utcnow()
last_month = today - timedelta(days=30)

# change the datetime foramt as ISO 8601
since = last_month.isoformat() + 'Z'
until = today.isoformat() + 'Z'

# request the number of Commits
params = {'since': since, 'until': until}
commit_count = count_commits(owner, repo, params)
print(f"Commits in the last day: {commit_count}")

# request the number of Issues
issue_count = count_issues(owner, repo, params)
print(f"Issues in the last day: {issue_count}")

# request the number of Pull Requests
pull_count = count_pull_requests(owner, repo, params)
print(f"Pull Requests in the last day: {pull_count}")

# request the number of Watchers

before_month_watcher_params = {"since": since, "until": since}
before_month_watcher_count = count_watcher(owner, repo, before_month_watcher_params)

after_month_watcher_params = {"since": until, "until": until}
after_month_watcher_count = count_watcher(owner, repo, after_month_watcher_params)

watcher_diff = after_month_watcher_count - before_month_watcher_count
print(f"Watchers in the last day: {watcher_diff}")

Commits in the last day: 0
Issues in the last day: 19
Pull Requests in the last day: 30
Watchers in the last day: 0


In [42]:
system_template = """
You should rate the project based on the following criteria, 
but at this time, only the github information will be suggested. 
and if the socre is less than 15, you should consider the project as dead.
Unless, you should consider the project as alive.
and you should make the report based on your own opinion for the project. 
:

### scoring criteria

1. GitHub Score (30 points total)

Frequency of commits:

- more than 20 commits within last month : 12 points
- more than 10 commits within last month  : 9 points
- Less than three commits within last month: 3 points
- No commits for more than 1 months: 0 points

Number of increased watchers:

- 5 or more increased : 9 points
- 3-4 increased : 6 points
- 1-2 increased : 3 points
- Not increased : 0 points

Issues and pull request activity:

- Frequent activity (discussions, quick resolutions within 1-2 weeks): 9 points
- Moderate activity (some unresolved issues, resolutions taking longer): 6 points
- Minimal activity (many unresolved issues, responses taking months): 3 points
- No issue or pull request activity: 0 points

### data
{data}
"""

data_input = f"""
commits in last month: {commit_count}
issues in last month: {issue_count}
pull request activity: {pull_count}
increased watchers: {watcher_diff}

"""

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{data}")]
)

In [50]:
chain = prompt_template | model | parser

result = chain.invoke({"data": data_input})

In [51]:
print(result)

Based on the given data, I would score the project as follows:

Frequency of commits: No commits for more than 1 month, so 0 points.

Number of increased watchers: No increased watchers, so 0 points.

Issues and pull request activity: Given the number of issues and pull request activity, it seems there is frequent activity. However, without knowing the speed of resolution, it's difficult to accurately score this. If we assume quick resolutions, this would be 9 points. 

Therefore, the total GitHub score for this project is 9 points. Since this score is less than 15, I would consider the project as dead. However, the high degree of issues and pull request activity does suggest that there may still be interest and activity within the project, so it might be worth investigating further before making a final decision.
